In [41]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
# Create lagged dataset
def create_lagged_dataset(dataset, lag, target_var):
    lagged_dataset = dataset.copy()
    columns_list = list(lagged_dataset.columns)
    data_join = {}
    for column in columns_list:
        if column == target_var:
            data_join[column] = lagged_dataset[column]
        for n in range(1,lag+1):
            data_join[F'{column}_L{n}'] = lagged_dataset[column].shift(n)
    lagged_dataset = pd.concat(data_join.values(), axis=1, ignore_index = True)
    lagged_dataset.columns = data_join.keys()
    return lagged_dataset.dropna() # dropping of NaN values so that regression can be easily done

# Generate Stepped Dataset for Training
## Steps is the number of weeks ahead that we are forecasting, e.g. step=2 is 2 weeks ahead.
## Note step=1 results in no change to dataset, i.e. use generated lagged variables to forecast current. 
def create_stepped_dataset(dataset, step, target_var):
    shifted_dataset = dataset.copy()
    y = shifted_dataset[[target_var]].shift(-step+1)
    if step != 1:
        X = shifted_dataset.iloc[:-step+1, :] # remove the last few rows
    else:
        X = shifted_dataset
    return X.drop(target_var, axis = 1), y.dropna()
## So now target variable (y variable for exploration) is shifted back by 2 weeks. i.e., taking the y-value from 2 weeks later
## and setting it to the current index. So linear regression of y+2 with the current X values. X will have
## a smaller dataset with the last 2 time points removed because of the shift. 

def generate_next_date(list_of_dates, date):
    return list_of_dates[list_of_dates > date].min()

def process_factor_model(X_factor, y_factor, train_end):
    X_train = X_factor.iloc[:train_end]
    X_test = X_factor.iloc[train_end:train_end+1]

    y_train = y_factor.loc[X_train.index]
    y_test = y_factor.loc[X_test.index]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, SGDRegressor
import numpy as np
import math
import keras_tuner as kt
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam

df = pd.read_csv("us_infl_final.csv")
lagged_df = create_lagged_dataset(df,lag=3,target_var="infl_rate")
covariates, y = create_stepped_dataset(lagged_df, step = 2, target_var = 'infl_rate')

def overall_function(dataset, outcome_var, lag_num, horizon_num):
    dataset['date'] = pd.to_datetime(dataset['date']) # converting to date format
    dataset = dataset.sort_values(by='date')
    dataset = dataset.drop(columns = 'date')
    # Creating Lagged and Stepped Datasets
    X_dataset, y_dataset = create_stepped_dataset(create_lagged_dataset(dataset, lag = lag_num, target_var = outcome_var), step=horizon_num, target_var = outcome_var)
    percentile_70 = math.ceil(len(X_dataset)*0.7) # 70-30 split
    
    # Initialising the training end
    train_end = percentile_70
    
    results = []
    
    ## Implement cross-validation split
    tscv = TimeSeriesSplit(n_splits = 3)
    
    while train_end < len(X_dataset):
        
        # Process data for modeling
        X_train, X_test, y_train, y_test = process_factor_model(X_dataset, y_dataset, train_end)
        
        # For Adding Results
        df_in_loop = y_test.copy()
        
        ### Model 1: Ridge Regression ###
        ridge_cv = RidgeCV(cv = tscv)
        ridge_cv.fit(X_train, y_train)
    
        ridge_model = Ridge(alpha = ridge_cv.alpha_)
        ridge_model.fit(X_train, y_train)
        
        ridgepred = ridge_model.predict(X_test)
        # Adding Linear Model
        df_in_loop[f'ridge'] = ridgepred
        print('Ridge Done')
        
        ### Model 2: Lasso Regression ###
        lasso_cv = LassoCV(cv = tscv)
        y = y_train.values.ravel()
        lasso_cv.fit(X_train, y)
    
        lasso_model = Lasso(alpha = lasso_cv.alpha_, max_iter=200000)
        lasso_model.fit(X_train, y)
        
        lassopred = lasso_model.predict(X_test)
        
        df_in_loop[f'lasso'] = lassopred
        print('Lasso Done')
        
        ## Model 3: LSTM ###
        X_train_lstm = np.array(X_train).astype(np.float32)
        X_test_lstm = np.array(X_test).astype(np.float32)
        y_train_lstm = np.array(y_train).astype(np.float32)
        
        # Define LSTM Model
        lstm_model = Sequential()
        # Layer 1
        lstm_model.add(LSTM(units = 128, return_sequences=True,input_shape=(X_train_lstm.shape[1], 1), activation='tanh'))
        lstm_model.add(Dropout(0.1))
        # Layer 2
        lstm_model.add(LSTM(units = 32, return_sequences=True, activation='tanh'))
        lstm_model.add(Dropout(0.1))
        # Layer 3 
        lstm_model.add(LSTM(units = 32, return_sequences=True, activation='tanh'))
        lstm_model.add(Dropout(0.2))
        # Layer 3 (Final Layer)
        lstm_model.add(LSTM(units = 32, return_sequences=False, activation='tanh'))
        lstm_model.add(Dropout(0.2))
        # Output Layer
        lstm_model.add(Dense(units=1))
        lstm_model.compile(
            optimizer=Adam(learning_rate=0.005),
            loss='mean_squared_error',
            metrics=[
                tf.keras.metrics.RootMeanSquaredError()
            ]
        )

        # Early stopping callback
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

        # Train the model
        lstm_model.fit(
            X_train_lstm, y_train_lstm,
            epochs=10, batch_size=512,
            callbacks=[callback]
        )

        lstmpred = lstm_model.predict(X_test_lstm)
        df_in_loop[f'lstm'] = lstmpred
        print('LSTM Done')
        
        # Add results into loop
        results.append(df_in_loop)
        train_end += 1
        num_remaining_dates = len(X_dataset) - train_end
        print(f'{num_remaining_dates} dates remaining')
        
        
    combined_df = pd.concat(results, ignore_index=True)
    return combined_df

c = overall_function(df, outcome_var = "infl_rate", lag_num = 2, horizon_num = 12)

Ridge Done
Lasso Done
Adaptive Lasso Done


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.6041324055080164e-07, tolerance: 4.732022720581692e-08
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.5761583363970884e-07, tolerance: 4.732022720581692e-08
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: D

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 8.9097e-05 - root_mean_squared_error: 0.0094
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 0.0191 - root_mean_squared_error: 0.1382
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 1.6974e-04 - root_mean_squared_error: 0.0130
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.0022 - root_mean_squared_error: 0.0472
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 768ms/step
LSTM Done
73 dates remaining
Ridge Done
Lasso Done


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.378934212622182e-08, tolerance: 4.7239608321568394e-08
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0375044317293075e-07, tolerance: 4.7239608321568394e-08
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:68

Adaptive Lasso Done
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 4.6600e-05 - root_mean_squared_error: 0.0068
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 0.0137 - root_mean_squared_error: 0.1170
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 1.4178e-04 - root_mean_squared_error: 0.0119
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 0.0011 - root_mean_squared_error: 0.0338
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 772ms/step
LSTM Done
72 dates remaining
Ridge Done
Lasso Done

...

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - loss: 2.1526e-05 - root_mean_squared_error: 0.0046
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 0.0070 - root_mean_squared_error: 0.0837
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 1.2676e-04 - root_mean_squared_error: 0.0113
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 7.0130e-04 - root_mean_squared_error: 0.0265
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 780ms/step
LSTM Done
2 dates remaining
Ridge Done
Lasso Done
Adaptive Lasso Done


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 6.0059e-05 - root_mean_squared_error: 0.0077
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 0.0062 - root_mean_squared_error: 0.0787
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 4.3728e-04 - root_mean_squared_error: 0.0209
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - loss: 9.5453e-04 - root_mean_squared_error: 0.0309
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 788ms/step
LSTM Done
1 dates remaining
Ridge Done
Lasso Done
Adaptive Lasso Done


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 1.7044e-04 - root_mean_squared_error: 0.0131
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 0.0082 - root_mean_squared_error: 0.0906
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 9.9509e-05 - root_mean_squared_error: 0.0100
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.0014 - root_mean_squared_error: 0.0378
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - loss: 6.0503e-04 - root_mean_squared_error: 0.0246
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 1.1452e-04 - root_mean_squared_error: 0.0107
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 796ms/step
LSTM Done
0 dates remaining


In [ ]:
c

,infl_rate,ridge,lasso,lstm
0,-0.000815,0.001023,0.001692,-0.049438
1,0.003001,0.001108,0.001678,-0.032881
2,0.003782,0.001736,0.001685,-0.027027
3,0.003760,0.001533,0.001697,0.029932
4,0.000247,0.001398,0.001708,0.014930
...,...,...,...,...
69,0.002265,0.002181,0.002116,-0.027590
70,0.002805,0.002194,0.002117,-0.051815
71,0.003647,0.002093,0.002120,-0.025671
72,0.004669,0.002178,0.002126,-0.016975
